In [1]:
import geopandas
import scipy

In [2]:

import datetime as dt
import json
import time
import boto3
import pandas as pd
from enricher import join_to_poi

In [3]:
kinesis = boto3.client('kinesis', region_name='us-east-2')


In [4]:

def get_shard_iterator(stream_name, shard_id, iterator_type, date):
    shard_response = kinesis.get_shard_iterator(
        StreamName=stream_name, 
        ShardId=shard_id, 
        ShardIteratorType=iterator_type, 
        Timestamp=date
    )

    return shard_response['ShardIterator']


In [5]:
#Our listings consumer will connect to kinesis, then periodically(we set this to 5 seconds) check for new data

def listings_consumer(next_shard_iterator):
    start = dt.datetime.now()
    finish = start + dt.timedelta(seconds=60)
    
    # we'll only consume for 60 seconds
    while start < finish:
        response = kinesis.get_records(
            ShardIterator=next_shard_iterator,
            Limit=20
        )
        
        try:
            data = response['Records'][0]['Data']
            data = json.loads(data)

            df = pd.DataFrame(data) 
            
#             print(df.columns)
            df = join_to_poi(df, k=1)
            df.to_csv('data.csv')
        except IndexError:
            # this error occurs when all records have been consumed so far
            print("No new records have arrived")
        except:
            print("Couldn't get shard id")
 # get the next shard iterator ie. where next to get data
        next_shard_iterator = response['NextShardIterator']
        
# pause checking for new data every 5 seconds
        time.sleep(1)
    

In [7]:
if __name__ == '__main__':
    stream_name =  'blossom-data-eng-emily'
    shard_id = 'shardId-000000000002'
    iterator_type = 'AT_TIMESTAMP'
    date = dt.datetime.today().date().__str__()
    
    next_shard_iterator = get_shard_iterator(stream_name, shard_id, iterator_type, date)
    listings_consumer(next_shard_iterator)
    

No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have 

ExpiredIteratorException: An error occurred (ExpiredIteratorException) when calling the GetRecords operation: Iterator expired. The iterator was created at time Tue Nov 26 21:21:50 UTC 2019 while right now it is Tue Nov 26 22:19:24 UTC 2019 which is further in the future than the tolerated delay of 300000 milliseconds.